<a href="https://colab.research.google.com/github/iam-fern/Research/blob/main/ex_sampling_100rep_n50_x6_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import statistics as stats
import scipy.stats as ss
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n50_x6_sas.csv')
df

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,38.074881,47.501957,29.305364,2.163909e+09,2.163909e+09,-1.415257e+08,2.305435e+09
1,0.0,1.0,47.722055,25.548748,34.240754,8.946889e+09,8.946890e+09,-9.591188e+06,8.956481e+09
2,0.0,2.0,32.936625,33.592004,19.476092,-1.685459e+08,-1.685455e+08,-1.340543e+08,-3.449127e+07
3,0.0,3.0,28.597172,34.693307,38.034853,4.685067e+08,4.685071e+08,2.007092e+07,4.484362e+08
4,0.0,4.0,32.452230,49.545322,10.918857,-1.393777e+09,-1.393776e+09,-2.861253e+08,-1.107651e+09
...,...,...,...,...,...,...,...,...,...
4995,99.0,45.0,40.855659,34.206762,39.727211,4.949556e+09,4.949557e+09,5.735301e+06,4.943821e+09
4996,99.0,46.0,49.488871,11.873135,10.970369,2.864648e+09,2.864649e+09,-1.285755e+08,2.993224e+09
4997,99.0,47.0,47.625297,10.268319,46.107398,1.325001e+10,1.325002e+10,1.669600e+08,1.308306e+10
4998,99.0,48.0,11.232647,24.371463,29.852099,1.290407e+06,1.290578e+06,5.220381e+07,-5.091323e+07


In [3]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n50_x6_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    38.074881  47.501957  ... -1.415257e+08  2.305435e+09
     1.0    47.722055  25.548748  ... -9.591188e+06  8.956481e+09
     2.0    32.936625  33.592004  ... -1.340543e+08 -3.449127e+07
     3.0    28.597172  34.693307  ...  2.007092e+07  4.484362e+08
     4.0    32.452230  49.545322  ... -2.861253e+08 -1.107651e+09
...               ...        ...  ...           ...           ...
99.0 45.0   40.855659  34.206762  ...  5.735301e+06  4.943821e+09
     46.0   49.488871  11.873135  ... -1.285755e+08  2.993224e+09
     47.0   47.625297  10.268319  ...  1.669600e+08  1.308306e+10
     48.0   11.232647  24.371463  ...  5.220381e+07 -5.091323e+07
     49.0   40.629392  44.215701  ... -2.237210e+08  6.557172e+08

[5000 rows x 7 columns]

In [4]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    38.074881  47.501957  ... -1.415257e+08  2.305435e+09
     1.0    47.722055  25.548748  ... -9.591188e+06  8.956481e+09
     2.0    32.936625  33.592004  ... -1.340543e+08 -3.449127e+07
     3.0    28.597172  34.693307  ...  2.007092e+07  4.484362e+08
     4.0    32.452230  49.545322  ... -2.861253e+08 -1.107651e+09
...               ...        ...  ...           ...           ...
99.0 45.0   40.855659  34.206762  ...  5.735301e+06  4.943821e+09
     46.0   49.488871  11.873135  ... -1.285755e+08  2.993224e+09
     47.0   47.625297  10.268319  ...  1.669600e+08  1.308306e+10
     48.0   11.232647  24.371463  ...  5.220381e+07 -5.091323e+07
     49.0   40.629392  44.215701  ... -2.237210e+08  6.557172e+08

[5000 rows x 7 columns]

In [5]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,38.074881,47.501957,29.305364,2.163909e+09,2.163909e+09,-1.415257e+08,2.305435e+09
1,0.0,1.0,47.722055,25.548748,34.240754,8.946889e+09,8.946890e+09,-9.591188e+06,8.956481e+09
2,0.0,2.0,32.936625,33.592004,19.476092,-1.685459e+08,-1.685455e+08,-1.340543e+08,-3.449127e+07
3,0.0,3.0,28.597172,34.693307,38.034853,4.685067e+08,4.685071e+08,2.007092e+07,4.484362e+08
4,0.0,4.0,32.452230,49.545322,10.918857,-1.393777e+09,-1.393776e+09,-2.861253e+08,-1.107651e+09
...,...,...,...,...,...,...,...,...,...
4995,99.0,45.0,40.855659,34.206762,39.727211,4.949556e+09,4.949557e+09,5.735301e+06,4.943821e+09
4996,99.0,46.0,49.488871,11.873135,10.970369,2.864648e+09,2.864649e+09,-1.285755e+08,2.993224e+09
4997,99.0,47.0,47.625297,10.268319,46.107398,1.325001e+10,1.325002e+10,1.669600e+08,1.308306e+10
4998,99.0,48.0,11.232647,24.371463,29.852099,1.290407e+06,1.290578e+06,5.220381e+07,-5.091323e+07


In [6]:
table.set_index(['rep', 'index'], inplace=True)

In [7]:
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    38.074881  47.501957  ... -1.415257e+08  2.305435e+09
     1.0    47.722055  25.548748  ... -9.591188e+06  8.956481e+09
     2.0    32.936625  33.592004  ... -1.340543e+08 -3.449127e+07
     3.0    28.597172  34.693307  ...  2.007092e+07  4.484362e+08
     4.0    32.452230  49.545322  ... -2.861253e+08 -1.107651e+09
...               ...        ...  ...           ...           ...
99.0 45.0   40.855659  34.206762  ...  5.735301e+06  4.943821e+09
     46.0   49.488871  11.873135  ... -1.285755e+08  2.993224e+09
     47.0   47.625297  10.268319  ...  1.669600e+08  1.308306e+10
     48.0   11.232647  24.371463  ...  5.220381e+07 -5.091323e+07
     49.0   40.629392  44.215701  ... -2.237210e+08  6.557172e+08

[5000 rows x 7 columns]

In [8]:
def get_result(run_replicate, percent_data):
    new_table = table.loc[run_replicate]

    # เรียง data จากน้อยไปมากตาม x1
    new_table_sort = new_table.sort_values('x1')

    new_table_sort.reset_index(inplace=True)

    #new_table_sort.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลกลุ่มที่ 1
    table1 = new_table_sort.iloc[:int(percent_data*new_table_sort.shape[0]) , :]
    # define respones variable
    y = table1['Y']
    # define predictor variable
    x = table1[['x1', 'x2', 'x3']]
    # add constant to predictor variable
    x = sm.add_constant(x)
    # fit linear regression model
    model1 = sm.OLS(y, x).fit()

    # หาค่า df กลุ่ม 1
    DF1 = model1.df_resid
    # หาค่า SSE1 : Residual sum of squares
    SSE1 = model1.ssr
    # หาค่า MSE1
    MSE1 = model1.mse_resid


    # แบ่งข้อมูลกลุ่มที่ 2
    table2 = new_table_sort.iloc[int((1-percent_data)*new_table_sort.shape[0]): , :]

    y = table2['Y']
    x = table2[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model2 = sm.OLS(y, x).fit()
    #print(model.summary())

    # หาค่า df กลุ่ม 2
    DF2 = model2.df_resid
    # หาค่า SSE2 : : Residual sum of squares
    SSE2 = model2.ssr
    # หาค่า MSE2
    MSE2 = model2.mse_resid

    # ค่าสถิติทดสอบ
    GQ_test = MSE2 / MSE1


    df2 = DF2
    df1 = DF1

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test, df2, df1)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test, df2, df1)
    
    return df2, df1, SSE2, SSE1, MSE2, MSE1, GQ_test, two_tailed, p_greater
    

    # เปรียบเทียบค่า p-value กับค่า alpha
    # Homoscedasticity : 0
    # Heteroscedasticity : 1
    #if p_value < alpha:
        #result_test='Reject005=0 : Heteroscedasticity '
        #result_test01=1
    #else:
        #result_test='Reject005=1 : Homoscedasticity '
        #result_test01=0
    
    #return df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test, result_test, p_value, result_test01


# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df1=[]
list_df2=[]
list_SSE1=[]
list_SSE2=[]
list_MSE1=[]
list_MSE2=[]
list_GQ_test=[]
#list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
#list_result_test01=[]


# กำหนด % การแบ่งข้อมูล
percent_data=0.4

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    #DF1, DF2, SSE1, SSE2, M_SSE1, M_SSE2, GQ_test, result_test=get_result(run_replicate, percent_data)
    df1, df2, SSE1, SSE2, MSE1, MSE2, GQ_test, two_tailed, p_greater=get_result(run_replicate, percent_data)
    list_df1.append(df1)
    list_SSE1.append(SSE1)
    list_df2.append(df2)
    list_SSE2.append(SSE2)
    list_MSE1.append(MSE1)
    list_MSE2.append(MSE2)
    list_GQ_test.append(GQ_test)
    #list_result_test.append(result_test)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)
    #list_result_test01.append(result_test01)

In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test":list_GQ_test, "two_tailed":list_two_tailed, "greater":list_p_greater})
result.set_index(['Replicate'], inplace=True)
result=result[(result["two_tailed"]<=1) & (result["greater"]<=1)]
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")

result.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,16.0,16.0,1.2030237498948557e+21,3.2519904506271763e+17,7.518898436842848e+19,2.0324940316419852e+16,3699.3458872637266,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
1.0,16.0,16.0,6.055623326410648e+20,4.087868182051216e+17,3.784764579006655e+19,2.55491761378201e+16,1481.364627411261,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
2.0,16.0,16.0,4.5947088335216476e+20,5.1188901791423176e+16,2.8716930209510298e+19,3199306361963948.5,8975.986342202603,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
3.0,16.0,16.0,1.540123682960231e+21,5.822649784732271e+17,9.625773018501444e+19,3.6391561154576696e+16,2645.056357328293,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
4.0,16.0,16.0,6.232809421013566e+20,4.3026478999557416e+16,3.895505888133479e+19,2689154937472338.5,14485.985295421637,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
5.0,16.0,16.0,6.767831169825743e+20,8.045648713605589e+16,4.229894481141089e+19,5028530446003493.0,8411.790535151014,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
6.0,16.0,16.0,1.9240955061730701e+21,3.9672929871902125e+17,1.2025596913581688e+20,2.4795581169938828e+16,4849.895161223743,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
7.0,16.0,16.0,8.607342121017575e+20,4.498567741617262e+17,5.379588825635984e+19,2.811604838510789e+16,1913.3516744427602,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
8.0,16.0,16.0,1.3535076652793001e+21,1.7444669879637344e+17,8.459422907995626e+19,1.090291867477334e+16,7758.860870501254,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity


In [10]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
result=pd.DataFrame({"Replicate":list_replicate, "df1":list_df1, "df2":list_df2, "SSE1":list_SSE1,
                     "SSE2":list_SSE2, "MSE1":list_MSE1, "MSE2":list_MSE2,
                     "GQ_test":list_GQ_test, "two_tailed":list_two_tailed, "greater":list_p_greater})
result.set_index(['Replicate'], inplace=True)

result['two_tailed_new']=result[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
result["Result_test"]=result["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
result_show=result.drop(columns=["two_tailed", "greater"])
result_show.astype(str)

,df1,df2,SSE1,SSE2,MSE1,MSE2,GQ_test,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,16.0,16.0,1.2030237498948557e+21,3.2519904506271763e+17,7.518898436842848e+19,2.0324940316419852e+16,3699.3458872637266,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
1.0,16.0,16.0,6.055623326410648e+20,4.087868182051216e+17,3.784764579006655e+19,2.55491761378201e+16,1481.364627411261,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
2.0,16.0,16.0,4.5947088335216476e+20,5.1188901791423176e+16,2.8716930209510298e+19,3199306361963948.5,8975.986342202603,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
3.0,16.0,16.0,1.540123682960231e+21,5.822649784732271e+17,9.625773018501444e+19,3.6391561154576696e+16,2645.056357328293,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
4.0,16.0,16.0,6.232809421013566e+20,4.3026478999557416e+16,3.895505888133479e+19,2689154937472338.5,14485.985295421637,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
5.0,16.0,16.0,6.767831169825743e+20,8.045648713605589e+16,4.229894481141089e+19,5028530446003493.0,8411.790535151014,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
6.0,16.0,16.0,1.9240955061730701e+21,3.9672929871902125e+17,1.2025596913581688e+20,2.4795581169938828e+16,4849.895161223743,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
7.0,16.0,16.0,8.607342121017575e+20,4.498567741617262e+17,5.379588825635984e+19,2.811604838510789e+16,1913.3516744427602,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
8.0,16.0,16.0,1.3535076652793001e+21,1.7444669879637344e+17,8.459422907995626e+19,1.090291867477334e+16,7758.860870501254,2.220446049250313e-16,Reject005=0 : Heteroscedasticity


In [11]:
result_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    100
Name: Result_test, dtype: int64